In [1]:
#ref: WorkSheet 7 & 8
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd
import re
import csv
rootpath='./'
train_path = './train'
test_path = './test'

In [2]:
train_dir = [i for i in os.listdir("train") if re.match(r'(.*?.jpg)',i)]
test_dir = [i for i in os.listdir("test") if re.match(r'(.*?.jpg)',i)]

In [3]:
sift = cv2.SIFT_create()

In [4]:
def sift_res(x_dir,x_path):
    kps = {}
    dess = {}
    for i in tqdm(x_dir):
        target_img = cv2.imread(os.path.join(x_path,i),0)
        kp, des =sift.detectAndCompute(target_img,None)
        kps[i] = kp
        dess[i] = des
    return kps,dess
kps_sift_train, dess_sift_train = sift_res(train_dir,train_path)
kps_sift_test, dess_sift_test = sift_res(test_dir,test_path)

  1%|          | 70/7500 [00:04<07:41, 16.09it/s]


KeyboardInterrupt: 

In [ ]:
def find_top_n(img_path, target,num):
    res = {}
    img = cv2.imread(os.path.join(test_path,img_path),cv2.IMREAD_GRAYSCALE)
    kp1 = kps_sift_test[img_path]
    des1 = dess_sift_test[img_path]
    for i in target:
        target_img = cv2.imread(os.path.join(train_path,i),cv2.IMREAD_GRAYSCALE)
        kp2 = kps_sift_train[i]
        des2 = dess_sift_train[i]
        # FLANN parameters and initialize
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        if(len(kp1)>=2 and len(kp2)>=2):
            # Matching descriptor using KNN algorithm
            matches = flann.knnMatch(des1, des2, k=2) 
            # Create a mask to draw all good matches
            matchesMask = []
            # Store all good matches as per Lowe's Ratio test.
            good = []
            for m,n in matches:
                if m.distance < 0.7*n.distance:
                    good.append(m)
                    matchesMask.append([1,0]) # Match
                else:
                    matchesMask.append([0,0]) # Mismatch
            # Print total number of good matches between two images
            res[i] = len(good)/len(matchesMask)
    return sorted(res.items(), key = lambda x:(x[1],x[0]), reverse=True)[:num]

In [ ]:
def find_best(top_n,img_path):
    ret = []
    img = cv2.imread(os.path.join(test_path,img_path),0)
    kp1 = kps_sift_test[img_path]
    des1 = dess_sift_test[img_path]
    for i in top_n:
        candidate = cv2.imread(os.path.join(train_path,i[0]),0)
        kp2 = kps_sift_train[i[0]]
        des2 = dess_sift_train[i[0]]
        # FLANN parameters and initialize
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        # Matching descriptor using KNN algorithm
        matches = flann.knnMatch(des1,des2,k=2)
        # Create a mask to draw all good matches
        matchesMask = []
        # Store all good matches as per Lowe's Ratio test.
        good = []
        for m,n in matches:
            if m.distance < 0.7*n.distance:
                good.append(m)
                matchesMask.append([1,0]) # Match
            else:
                matchesMask.append([0,0]) # Mismatch
        MIN_MATCH_NUM = 4
        if len(good)> MIN_MATCH_NUM:

            ptsA = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
            ptsB = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
            H, status = cv2.findHomography(ptsA, 
                                           ptsB, 
                                           cv2.RANSAC, 
                                           ransacReprojThreshold = 5, 
                                           maxIters = 10) # try to change maxIters and see the effect
            success = status.ravel().tolist()
            #Threshold to avoid match wall
            if(success.count(1)/len(good) > 0.15):
                ret.append(i[0])
                if len(ret)>1:
                    return ret
    if len(ret)<1 and len(top_n)>1:
        ret.append(top_n[0][0])
        ret.append(top_n[1][0])
    return ret

In [ ]:
train_csv = pd.read_csv("train.csv")

def main_func(flann_num=50,start=0,end=1200):
    errors = []
    for test_file in tqdm(test_dir[start:end]):
        flann_top = find_top_n(test_file,train_dir,flann_num)
        best_match = find_best(flann_top,test_file)
        if len(best_match)==1:
            best_match = re.sub(".jpg","",best_match[0])
            x = train_csv.loc[train_csv["id"]==best_match]["x"].values[0]
            y = train_csv.loc[train_csv["id"]==best_match]["y"].values[0]
            f = open('test_sift.csv','a')
            writer = csv.writer(f)
            test_file = re.sub(".jpg","",test_file)
            writer.writerow((test_file,x,y,best_match,0))
            f.close()
        elif len(best_match)==2:
            best_match1 = re.sub(".jpg","",best_match[1])
            best_match = re.sub(".jpg","",best_match[0])
            x = train_csv.loc[train_csv["id"]==best_match]["x"].values[0]
            y = train_csv.loc[train_csv["id"]==best_match]["y"].values[0]
            f = open('test_sift.csv','a')
            writer = csv.writer(f)
            test_file = re.sub(".jpg","",test_file)
            writer.writerow((test_file,x,y,best_match,best_match1))
            f.close()
        else:
            errors.append(test_file)
            print(test_file)
    return errors

In [ ]:
err = main_func(start=0,end=100)